### Distribution Transformation

Distribution transformation is a very useful tool which will be extensively used with the copula concept that we discuss in the next Section.
The technique transforms every random variables from uniform to any distribution and vice versa and is called \emph{probability integral transform} or (percentile-to-percentile transform). 

Computationally, this method involves computing the quantile function of the distribution, in other words, computing the cumulative distribution function (CDF) of the distribution (which maps a number in the domain to a probability between 0 and 1) and then inverting that function. We won't go into the details but we will just show few examples of how this can be done in $\tt{python}$.

For example, imagine that $\mathbb{P}(X)$ is the standard normal distribution with mean zero and standard deviation one. If we want to convert uniformly distributed samples to standard normal we need to apply the inverse of CDF to each sample. Below few examples which exploit $\tt{scipy.stats}$ which defines many useful statistical distributions.

In [22]:
from scipy.stats import uniform, norm

uniform_samples = [0.5, .975, 0.995, 0.999999]

print ("Unif.\t\tStd. Normal")
for u in uniform_samples:
    print ("{:.7f}\t{:.8f}".format(u, norm.ppf(u)))


Unif.		Std. Normal
0.5000000	0.00000000
0.9750000	1.95996398
0.9950000	2.57582930
0.9999990	4.75342431


The same transformation may be applied directly on the entire sample.

In [43]:
from scipy import stats

x = stats.uniform(0, 1).rvs(10000)

<img src="lesson6_files/lesson6_3_0.png">

Next we apply $\tt{ppf()}$ to the list of $x$ directly.

In [44]:
norm = stats.norm() 
x_trans = norm.ppf(x)

<img src="lesson6_files/lesson6_5_0.png">

If we plot them togheter in a 2D plot we can get a sense of what is going on when using the inverse CDF transformation:

<img src="lesson6_files/lesson6_7_0.png">

The inverse CDF stretches the outer regions of the uniform to yield a normal distribution. 

The nice thing of the technique is that it can be used with any arbitrary (univariate) probability distributions, like for example [t-Student](https://en.wikipedia.org/wiki/Student%27s_t-distribution) or [Gumbel](https://en.wikipedia.org/wiki/Gumbel_distribution)

In [45]:
tstudent = stats.distributions.t(4)
x_trans = tstudent.ppf(x)

<img src="lesson6_files/lesson6_9_0.png">

Clearly to do the opposite transformation from an arbitray distribution to the uniform(0, 1) we can just apply the inverse of the inverse CDF, which is the CDF itself...

In [46]:
x = stats.uniform(0, 1).rvs(10000)
norm = stats.norm() 
x_trans = norm.ppf(x)
x_uniform = norm.cdf(x_trans)

<img src="lesson6_files/lesson6_11_0.png">

## Copula

In probability theory a *copula* $C(U_1, U_2, \ldots, U_n, \rho)$ is a multivariate (multidimensional) cumulative distribution function for which the marginal probability distribution (the probability distribution of each dimension) of each variable is uniform on the interval $[0, 1]$ ($U_i \approx$~Uniform(0,1)). $\rho$ represent the correlation between each variable. 

*Sklar's theorem* states that any multivariate joint distribution can be written in terms of univariate marginal distribution functions and a copula which describes the dependence structure between the variables.

Copulas are used to describe the dependence between random variables and have been used widely in quantitative finance to model risk.

Despite the obscure and daunting definition the concept of copula is quite simple so let's try to clarify it a bit with a practical example.

### Example Problem Case
Imagine to measure two variables that are correlated. For example, we look at various rivers and for every river we look at its maximum water level, and also count how many months each river caused flooding. 

For the probability distribution of the maximum level of the river we know that maximums are Gumbel distributed, while the number of flooding can be modelled according to a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution).

Clearly it is pretty reasonable to assume that the maximum level and the number of floodings is going to be correlated, however we don't know how we could model that correlated probability distribution. Above we only specified the distributions for individual variables, irrespective of the other one (i.e. the marginals), in reality we are dealing with the joint distribution of both of these together. 

And here is where copulas come to our rescue.

Copulas essentially allow to decompose a joint probability distribution into their marginals (which by definition have no correlation) and a function which couples (hence the name) them together and thus allows us to specify the correlation separately. The copula is that coupling function.

## Adding Correlation with Gaussian Copulas
We are actually almost done indeed, we saw before how to convert anything uniformly distributed to an arbitrary probability distribution. So that means we need to generate uniformly distributed data with the correlation we want and then transform the marginals into the desired distributions. 

How do we do that ? 

* simulate from a multivariarte Gaussian with the specific corrrelation structure (since we know very well how to do it);
* transform each Gaussian marginal to uniform;
* finally transform the uniforms to whatever we like.

So let's sample from a multivariate normal (2D) with a 0.5 correlation.

In [47]:
mvnorm = stats.multivariate_normal(mean=[0, 0] , cov=[[1, 0.5],
                                                      [0.5, 1]])
x = mvnorm.rvs(100000)

<img src="lesson6_files/lesson6_14_0.png">

Now use what we have just seen to tranform the marginals to uniform using the $\tt{cdf}$ function of the normal distribution ($x$ is a 2D vector in this case, but tranformation will be applied separately on each component)

In [48]:
norm = stats.norm()
x_unif = norm.cdf(x)

<img src="lesson6_files/lesson6_3_0.png" width=300>
<img src="copula_3d.png" width=300>

These plots above is usually how copulas are visualized.

Finally we can just transform the marginals again from uniform to what we want (i.e. Gumbel and Beta in our river example): 

In [49]:
m1 = stats.gumbel_l()
m2 = stats.beta(a=10, b=3)

# transform U1 into Gumbel
# [:, 0] means all entries of dim 0  
x1_trans = m1.ppf(x_unif[:, 0])

#transform U2 into Beta
# [:, 1] means all entries of dim 1
x2_trans = m2.ppf(x_unif[:, 1])

<img src="lesson6_files/lesson6_18_0.png">

To see that it is actually working as expected we should now compare our scatter plot with correlation to the joint distribution of the same marginals without correlation:

In [50]:
# sample from Gumbel
x1 = m1.rvs(10000)
# sample from Beta
x2 = m2.rvs(10000)

<img src="lesson6_files/lesson6_20_0.png">

Using the uniform distribution as a common base for our transformations we can easily introduce correlations and flexibly construct complex probability distributions. Clearly this is directly extendeable to higher dimensional distributions as well.

### Generate Correlated Distributions
Let's now see how copulas can be used to generate numbers from correlated distributions. These are the steps to follow:

* generate a random vector $\mathbf{x}=(x_1, x_2,\ldots)$ from a multivariate distribution with the desired correlation;
* determine the single $U_i(x_i)$ by applying $\tt{cdf}$ to each $x_i$;
* transform again each $U_i(x_i)$ to the desired marginal distributions using $\tt{ppf}$.

Each component of the vector $\mathbf{x}$ is now transformed as it was drawn from the desired marginals with the appropriate correlation.

A practical application concerns the probability of default. Imagine there are three companies (A, B and C) which have a cumulative probability of defaulting within the next two years of 10%.

Let’s try to compute the probabilities to have the three of them all defaulting within the next two years in the cases with independent and correlated default probabilities.

In the first case (independent probabilities), the odds to get three defaults within two years is the product of the single probabilities, hence:
$$\mathbb{P}_{\mathrm{uncorr}}= 10\%\cdot 10\%\cdot 10\% = 0.1\%$$

We can verify this in $\tt{python}$ by applying the Monte Carlo algorithm outlined above: generate a random sample from an uncorrelated multivariate normal distribution, then transform each sample
( $[ x_A , x_B , x_C ]$ ) into the uniform distribution with the $\tt{norm.cdf}$ function (i.e. we convert the samples into probabilities) and then count how many times the three of them are lower than 10%. The final probability will be the ratio of the numer of successes by the total trials.

In [58]:
from scipy.stats import multivariate_normal, uniform, norm
import numpy

numpy.random.seed(10)
trials = 500000
mvnorm_no_corr = multivariate_normal(mean=[0, 0, 0], cov=[[1, 0, 0],
                                                          [0, 1, 0],
                                                          [0, 0, 1]])

defaults = 0
x = mvnorm_no_corr.rvs(trials)
x_trans = norm.cdf(x)
for i in range(len(x_trans)):
    if x_trans[i][0] < 0.1 and x_trans[i][1] < 0.1 and \
       x_trans[i][2] < 0.1:
        defaults += 1

print ("Defaults w/o correlation: {:.2f}%".format(defaults/trials*100))

Defaults w/o correlation: 0.10%


If we repeat the same Monte Carlo experiment with perfectly correlated default probabilities
we have

In [59]:
from scipy.stats import multivariate_normal, uniform, norm
import numpy

numpy.random.seed(10)
trials = 500000
mvnorm_no_corr = multivariate_normal(mean=[0, 0, 0], 
                                     cov=[[1, 0.99999, 0.99999],
                                          [0.99999, 1, 0.99999],
                                          [0.99999, 0.99999, 1]])

defaults = 0
x = mvnorm_no_corr.rvs(trials)
x_trans = norm.cdf(x)
for i in range(len(x_trans)):
    if x_trans[i][0] < 0.1 and x_trans[i][1] < 0.1 and \
       x_trans[i][2] < 0.1:
        defaults += 1

print ("Defaults w/o correlation: {:.2f}%".format(defaults/trials*100))

Defaults w/o correlation: 9.96%


In this case the result is 10%, like we had only one single company, indeed being perfectly
correlated either there is no default or three "simultaneous" defaults with 10% probability.

## Application to Finance
In credit derivative valuation and credit risk management, one of the most important issue is the estimate of default probabilities and their correlations. Default correlation measures the tendency of two companies to default at about the same
time. For this, generally speaking, there are two ways: using historical default data or using mathematical models, like copulas. 

Historical default data has played an important role in the estimation of default probabilities. However, because default events are rare, there is very limited default data available. Moreover, historical data reflects the historical default pattern only and it may not be a proper indicator of the future. This makes the estimation of default probabilities from historical data difficult and inexact. To use this same data to estimate default correlations is even more difficult and more inexact. 

On the other hand mathematical models don't rely on historical default data. In Chapter~\ref{} we have seen how it is possible to derive default probabilities from market data. Before going into the details of the application of the copula to default probabilities let's introduce two more kind of contracts.

## Basket Default Swaps
A basket default swap is a credit derivative on a portfolio of reference entities. The
simplest basket default swaps are first-to-default swaps, second-to-default swaps,
and nth-to-default swaps.

With respect to a basket of reference entities, a first-to-default swap provides insurance for only the first default, a second-to-default swap
provides insurance for only the second default, an nth-to-default swap provides
insurance for only the nth default. 

For example, in an nth-to-default swap, the protection seller does not make a payment to the protection buyer for the first n−1 defaulted reference entities, and makes the payment only for the nth defaulted reference entity. Once there is a payment upopn the default of the a defaulted reference entity, the swap terminates. 

### Calculating nth-to-default Probabilities

The valuation of a basket default swap comes down to the calculation of relevant default probabilities. So let's see how we can compute them.

#### Independent Defaults
If the default times of the names of a basket are independent, first-to-default, nth-to-default, all-to-default probabilities can be calculated through multiplication and integration of the default probability curves of each basket components. 

As an example, we consider the second-to-default probability of a 3-name basket. Let $\tau_i$ be the default time of name $i$ and $F_i(t)$ its distribution. Then the probability that name 1 defaults second in the basket before time $t$: 

$$
\begin{align*}
&P((\tau_2\lt\tau_1)\cap (\tau_1\lt t)\cap (\tau_1\lt\tau_3)) +
P((\tau_3\lt\tau_1)\cap (\tau_1\lt t)\cap (\tau_1\lt\tau_2)) = \\
&\int_0^t{F_2 (s)\cdot (1-F_3 (s))~dF_1(s)} +  \int_0^t{F_3 (s)\cdot (1-F_2 (s))~dF_1(s)}
\end{align*}
$$

The formula for nth-to-default probability in a general basket can be derived similarly, however, complexity increases as the number of names increases.

Suppose the default probabilities of three companies, A, B and C are given as in the following table:

|time in years | A | B | C |
| :-:|:-:|:-:|:-:|
|0 | 0 | 0 | 0 |
|1 | 0.022032 | 0.0317 | 0.035 |
|2 | 0.046242 | 0.0655 | 0.075 |
|3 | 0.07266 | 0.1022 | 0.121 |
|4 | 0.101233 | 0.142 | 0.153 |
|5 | 0.131885 | 0.1752 | 0.205 |

and suppose that the default events of the three companies are independent. 

The default probabilities are linear in each time interval so the integral above can be solved by substitution:

$$ \int_{x_0}^{x_1}{(1-F_B(x))(1-F_C(x))dF_A(x)}$$

Setting $t=m_A x + q_A$ it becomes with $m_A, q_A$ are the parameters of the line joining the default probabilities of company A:

$$ \int_{m_A x_0 + q_A}^{m_A x_1 + q_A}{(1-F_B(x(t)))(1-F_C(x(t)))dt}\qquad\textrm{, with}~x(t) = \cfrac{t -q_A}{m_A} $$
and similarly for company B and C.

To convert it into python we can use $\tt{scipy.integrate.quad}$ to perform the integral and $\tt{numpy.interp}$ to determine the intermediate default probabilities.

In [ ]:
from scipy.integrate import quad
from numpy import interp

default_rates = {"A":(0, 0.022032, 0.046242, 0.07266, 0.101233, 0.131885), # company A
                 "B":(0, 0.0317, 0.0655, 0.1022, 0.142, 0.1752), # company B
                 "C":(0, 0.035, 0.075, 0.121, 0.153, 0.205)} # company C

def func(x, default, companies, t):
    m = default[companies[0]][t] - default[companies[0]][t-1]
    q = default[companies[0]][t-1] - m * (t-1)
    t = (x-q)/m
    F2 = 1 - interp(t, range(len(default[companies[1]])), default[companies[1]])
    F3 = 1 - interp(t, range(len(default[companies[2]])), default[companies[2]])
    return F2*F3

def integral(default, companies, t):
    return quad(func, 0, default[companies[0]][t], args=(default, companies, t))[0]
                 
for companies in [("A", "B", "C"), ("B", "A", "C"), ("C", "A", "B")]:
    prob = 0
    for t in range(1, 6):
        prob = integral(default_rates, companies, t)
        print ("First to default prob at time ({}) for company {}: {:.5f}".format(t, companies[0], prob))


#### Correlated Defaults
When the default probabilities of the companies are correlated the copula approach can be used. 

Suppose we would like to simulate the defaults for the next 5 years for 10 companies. The copula default correlation between each company is 0.2 and the cumulative probability of default during the next 1,2,3,4 5 years is 1%, 3%, 6%, 10%, 13% respectively for each company.

When a Gaussian copula is used in order to simulate the defaults we need to sample from a multivariate normal distribution a vector $\mathbf{x}$, transform then each $x_i$ into the default time $t_i$.

In this simple example company 10 would have defaulted during first year ($\mathbb{P}(t_{10}) \le 1\%$, companies 8 and 9 during the second (both have $3\% \le \mathbb{P}(t_{8,9}) \le 6\%$) and so on.

To determine nth-to-default probabilities it is enough to repeat many sampling from the multivariate normal distributions and count how many times we have n defaults before each year.

In [21]:
from scipy.stats import multivariate_normal, norm

default_rates = [[0,0.022032,0.046242,0.07266,0.101233,0.131885],
                 [0,0.0317,0.0655,0.1022,0.142,0.1752],
                 [0,0.035,0.075,0.121,0.153,0.205],
                 [0,0.008,0.013,0.019,0.03,0.05]]

np.random.seed(1)
mvnorm = multivariate_normal(mean=[0, 0, 0, 0] , cov=[[1, 0.5, 0.7, 0.2],
                                                      [0.5, 1, 0.3, 0.25],
                                                      [0.7, 0.3, 1, 0.3],
                                                      [0.2, 0.25, 0.3, 1]])

tot = 1000000
x = mvnorm.rvs(size=tot)
x_unif = norm.cdf(x)

succ = [[0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.]]

nth_default = 1

for i in range(tot):
    x_s = sorted(x_unif[i])
    for c in range(4):
        if x_s[nth_default-1] == x_unif[i, c]:
            for y in range(6):
                temp = True
                if x_unif[i, c] < default_rates[c][y]:
                    succ[y][c] += 1.

for y in range(6):
     print(sum(succ[y]) / float(tot)),

0.0
0.076723
0.145731
0.213026
0.274272
0.342925


In [22]:

default_rates = [[0,0.022032,0.046242,0.07266,0.101233,0.131885],
                 [0,0.0317,0.0655,0.1022,0.142,0.1752],
                 [0,0.035,0.075,0.121,0.153,0.205],
                 [0,0.008,0.013,0.019,0.03,0.05]]

np.random.seed(1)
mvnorm = multivariate_normal(mean=[0, 0, 0, 0] , cov=[[1, 0, 0, 0],
                                                      [0, 1, 0, 0],
                                                      [0, 0, 1, 0],
                                                      [0, 0, 0, 1]])

tot = 1000000
x = mvnorm.rvs(size=tot)
x_unif = norm.cdf(x)

succ = [[0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.], 
        [0., 0., 0., 0.]]

nth_default = 1

for i in range(tot):
    x_s = sorted(x_unif[i])
    for c in range(4):
        if x_s[nth_default-1] == x_unif[i, c]:
            for y in range(6):
                temp = True
                if x_unif[i, c] < default_rates[c][y]:
                    succ[y][c] += 1.

for y in range(6):
     print(sum(succ[y]) / float(tot)),

0.0
0.092454
0.182155
0.271455
0.35076
0.437701


### Collateralized Debt Obligation
A collateralized debt obligation (CDO) is a security backed by a diversified pool of
one or more kinds of debt obligations such as bonds, loans, credit default swaps or
structured products (mortgage-backed securities, asset-backed securities, and even
other CDOs). A CDO can be initiated by one or more of the following: banks,
nonbank financial institutions, and asset management companies, is referred to as
the sponsor. The sponsor of a CDO creates a company so-called the special purpose
vehicle (SPV). The SPV works as an independent entity. In this way, CDO investors
are isolated from the credit risk of the sponsor. Moreover, the SPV is responsible
for the administration. The SPV obtains the credit risk exposure by purchasing
debt obligations (bonds or residential and commercial loans) or selling CDSs; it
transfers the credit risk by issuing debt obligations (tranches/credit-linked notes).
The investors in the tranches of a CDO have the ultimate credit risk exposure to
the underlying reference entities.
The SPV issues four kinds of tranches. Each tranche has
an attachment percentage and a detachment percentage. When the cumulative
percentage loss of the portfolio reaches the attachment percentage, investors
in the tranche start to lose their principal, and when the cumulative percentage loss
of principal reaches the detachment percentage, the investors in the tranche lose all
their principal and no further loss can occur to them.

In the literature, tranches of a CDO are classified as subordinate/equity tranche,
mezzanine tranches, and senior tranches according to their subordinate levels. 
Because the equity tranche is extremely
risky, the sponsor of a CDO holds the equity tranche and the SPV sells other tranches
to investors.

### Role of Correlation in Basket CDS and CDO
The cost of protection in a nth-to-default CDS or a trancehe of a CDO is critically dependent on default correlation. Suppose that a basket of a 1000 reference entities is used to define a 5-year nth-to-default CDS and that each reference entity has a probability of 2% of defaulting during the 5 years. When the default corralation between the reference entities is zero the binomial distribution (see Appendix~\ref{appendix1}) shows that the probability of one or more defaults during the 5 years is 86.7% and the probability of 10 or more defaults is 0.00034%. A first-to-default CDS is therefore quite valuable wereas a 10th-to-default CDS is worth almost nothing.

In [ ]:
from scipy.stats import binom

b = binom(100, 0.02)
print ("P(x>=1) : {:.4f}".format(b.cdf(100)-b.cdf(0)))
print ("P(x>=10): {:.6f}".format(b.cdf(100)-b.cdf(9)))

As the default correlation increases the probability of one or more defaults declines and the probability of 10 or more defaults increases. In the limit where the default correlation between the reference entities is perfect the probability of one or more defaults equals the probability of ten or more defaults and is 2%. This is because in this extreme situation the reference entities are essentially the same: either they all default (with probability 2%) or none of them default (with probability 98%).

### Standard Market Model
The critical input into pricing CDO tranches or Basket CDS is an estimate of the default dependence (default correlation) between the underlying assets. 

One popular method for estimating the dependence structure is using copula functions, a method first applied in actuarial science. While there are several types of copula function models, the first introduced was the one-factor Gaussian copula model. These models have the advantage that can be solved semianalyticly.

Consider a portfolio of $N$ companies and assume that the marginal probabilities of default are known for each company. Define:

* $t_i$, the time of default of the $i$th company:
* $Q_i(t)$, the cumulative probability that company $i$ will default before time $t$; that is, the probability that $t_i \le t$;
* $S_i(t) = 1 – Q_i(t)$, the probability that company $i$ will survive beyond time $t$; that is, the probability that $t_i > t$. 

To generate a one-factor model for the $t_i$ we define random variables $x_i$ $(1\le i \le N)$

$$X_i = a_i M + \sqrt{1-a_i^2 Z_i},\qquad i = 1, 2,\ldots, n$$

where $M$ and the $Z_i$ have independent zero-mean unit-variance distributions and
$–1 \le a_i \lt 1$. Equation (1) defines a correlation structure between the $x_i$ dependent on a single common factor $M$. The correlation between $x_i$ and $x_j$ is $a_i a_j$.
Let $F_i$ be the cumulative distribution of $x_i$. Under the copula model the $x_i$ are mapped to the $t_i$ using a *percentile-to-percentile* transformation. The five-percentile point in the probability distribution for $x_i$ is transformed to the five-percentile point in the probability distribution of $t_i$; the ten-percentile point in the probability distribution for $x_i$ is transformed to the ten-percentile point in the probability distribution of $t_i$; and so on. In general the point $x_i = x$ is transformed to $t_i = t$ where $t = Q_i^{–1}[F_i(x)]$.

Let $H$ be the cumulative distribution of the $Z_i$.
It follows from Equation (1) that 

$$\mathbb{P}(x_i < x|M) = H\left(\cfrac{x-a_i M}{\sqrt{1-a_i^2}}\right)$$

When $x = F_i^{–1}[Q_i(t)]$, $\mathbb{P}(t_i < t) = \mathbb{P}(x_i < x)$. Hence

$$\mathbb{P}(t_i < t|M) = H\left\{\cfrac{F_i^{–1}[Q_i(t)]-a_i M}{\sqrt{1-a_i^2}}\right\}$$

The conditional probability that the $i$th bond will survive beyond time T is therefore

$$S_i(T|M) = 1 - H\left\{\cfrac{F_i^{–1}[Q_i(t)]-a_i M}{\sqrt{1-a_i^2}}\right\}$$

The model we have presented can be extended to many factors quite easily but we won't do it in these lectures.

The advantage of the copula model is that it creates a tractable multivariate joint
distribution for a set of variables that is consistent with known marginal probability
distributions for the variables. One possibility is to let the $M$’s and the $Z$’s have standard
normal distributions. A Gaussian copula then results. However, any distributions can be
used for $M$’s and the $Z$’s (providing they are scaled so that they have zero mean and unit
variance). Each choice of distributions results in a different copula model. The choice of
the copula governs the nature of the default dependence. For example, as we will see,
copulas where the $M$’s have heavy tails generate models where there is a greater
likelihood of a clustering of early defaults for several companies. Later on we
will explore the effect of using normal and t-distributions for the $M$'s and $Z$'s. 

<!----Suppose that a CDO includes $n$ assets $i = 1, 2,\ldots, n$ and the default times $\tau_i$ of the $i$th asset has a default intensity $\lambda_i$. Then the probability of a default occurring before the time $t$ is
$$\mathbb{P}(\tau_i \lt t) = 1 - \mathrm{exp}(-\lambda_i t)$$
see Chapter~\ref{hazard}.----->

For simplicity, the following two assumptions are made:

* all the companies have the same default intensity, i.e, $\lambda_i = \lambda$;
* the pairwise default correlations are the same, i.e $a_i = a$.

The second assumption means that the contribution of the market component is
the same for all the companies and the correlation between any two companies is
constant, $\rho = a^2$.

Under these assumptions, given the market situation $M = m$, all the companies
have the same cumulative default probability $D_{t|M}=\mathbb{P}(t_i < t|M)$. Moreover, for a
given value of the market component $M$, the defaults are mutually independent for
all the underlying companies. Letting $N_{t|m}$ be the total defaults that have occurred
by time $t$ conditional on the market condition $M = m$, then $N_{t|m}$ follows a binomial
distribution, and

$$\mathbb{P}(N_{t|m} = j) = \cfrac{n!}{j!(n-j)!}D^j_{t|m}(1-D_{t|m})^{n-j},\qquad  j=0, 1, 2,\ldots,n$$
The probability that there will be exactly $j$ defaults by time $t$ is

$$\mathbb{P}(N_{t} = j) = \int_{-\infty}^{\infty}{\mathbb{P}(N_{t|m} = j)f_M(m)dm}$$

where $f_M(m)$ is the probability density function (PDF) of the random variable $M$.

Li (1999, 2000) was the first to suggest that the Gaussian copula could be employed
in credit risk modeling to estimate the correlation default. In a one-factor Gaussian
copula model, the distributions of the common market component $M$ and the individual component $Z_i$ are standard normal Gaussian distributions.
Because the sum of two independent Gaussian distributions is still a Gaussian distribution, the $X_i$ have a standard normal distribution.
The one-factor copula Gaussian copula model under the assumptions outlined above is the *market standard model*.

### Basket CDS Valuation under Market Standard Model

Consider a first-to-default basket. If the 1st-to-default probabilities are known, then the default probability of a basket of entities is simply the sum of the 1s-to-default probabilities of the individual entities in the basket. From this one can easily calculate the fair value of the premium payments of a basket default swap: if the protection is only for the first entity in the basket only the payoff value of the 1st defaulted entity needs to be calculated. 
For an nth-to-default swap similar arguments also hold. 

We now present some numerical results for an nth to default CDS. We assume that the
principals and expected recovery rates are the same for all underlying reference assets.
The valuation procedure is similar to that for a regular CDS where there is only one
reference entity.8
 In a regular CDS the valuation is based on the probability that a default
will occur between times T1 and T2. Here the valuation is based on the probability that the
n
th default will occur between times T1 and T2.
We assume the buyer of protection makes quarterly payments in arrears at a specified rate
until the n
th default occurs or the end of the life of the contract is reached. In the event of
an n
th default occurring the seller pays the notional principal times 1 – R. Also, there is a
final accrual payment by the buyer of protection to cover the time elapsed since the
previous payment. The contract can be valued by calculating the expected present value
of payments and the expected present value of payoffs in a risk-neutral world. The
breakeven CDS spread is the one for which the expected present value of the payments
equals the expected present value of payoffs.9
Consider first a 5-year n
th to default CDS on a basket of 10 reference entities in the
situation where the expected recovery rate, R, is 40%. The term structure of interest rates
is assumed to be flat at 5%. The default probabilities for the 10 entities are generated by
Poisson processes with constant default intensities, λi, (1 ≤ i ≤ 10) so that10 

10 CDS rho = 0.3 R = 40% rate = 5% 
Q(t) = [0.02, 0.0396, 0.0588, 0.0776, 0.0961]


In [47]:
from finmarkets import DiscountCurve, CreditCurve, CreditDefaultSwap
from finmarkets import GaussianQuadrature
from datetime import date
from dateutil.relativedelta import relativedelta
from scipy.stats import norm, binom
from math import sqrt, exp

n_cds = 10
rho = 0.6
l = 0.01
pillar_dates = []
df = []
today = date.today()
for i in range(6):
    pillar_dates.append(today + relativedelta(years=i))
    df.append(1/(1+0.05*i))
dc = DiscountCurve(today, pillar_dates, df)
gq = GaussianQuadrature()
Q = [1-exp(-(l*t)) for t in range(6)]
#Q = [0, 0.02, 0.0396, 0.0588, 0.0776, 0.0961]
values, weights = gq.M(60)
cds = CreditDefaultSwap(1, today, 0.01, 5)

for ndefault in range(1, 11):
    S = []
    
    for j in range(len(values)):
        temp = []
        for i in range(6):
            P = norm.cdf((norm.ppf(Q[i]) - sqrt(rho)*values[j])/
                         (sqrt(1-rho)))
            b = binom(n_cds, P)
            temp.append(1 - (b.cdf(n_cds)-b.cdf(ndefault-1)))
        S.append(temp)
    
    s = 0
    for j in range(len(values)):
        s += weights[j] * cds.breakevenRate(dc, 
                                            CreditCurve(pillar_dates, 
                                                        S[j]))

    print (s)

0.03214370055078652
0.015337587442302281
0.00725616955902986
0.004215577657531996
0.0022014564160901615
0.001225042309192564
0.0006383163501547495
0.0003071509287843383
0.00012742929384463388
3.669614592811156e-05


	0	0.3	0.6
1	153	181	321
2	25	51	153
3	3	18	73
4	0	7	42
5	0	3	22
6	0	0	12
7	0	0	6
8	0	0	3
9	0	0	1
10	0	0	0
			
			
	0.01	0.02	0.03
1	181	393	627
2	51	136	238
3	18	57	109
4	7	25	52
5	3	11	25
6	0	5	12
7	0	2	5
8	0	0	2
9	0	0	0
10	0	0	0

### Valuation of CDO
Suppose that the payment date on a CDO tranche are at times $\tau_i$. Define $\mathbb{E}_j$ the expected 
tranche principal at time $\tau$ and $D(\tau)$ the discount factor at time $\tau$. Suppose also that the spread
on a particular tranche (i.e. the number of basis point paid for protection on the remaining tranche principal) is $s$. 

The present value of the expected regular spread payments on the CDO is given by
\begin{equation}
s\cdot A = s\cdot \sum_{j=1}^{m}(\tau_j - \tau_{j-1})\mathbb{E}_{j}D(\tau_j)
\label{eq:A}
\end{equation}
The expected loss between times $\tau_{j-1}$ and $\tau_j$ is $\mathbb{E}_{j-1}-\mathbb{E}_j$. For simplicity assume
the loss occurs only at the midpoint of the time interval, so the present value of the expected payoffs on the CDO tranche is
\begin{equation}
C=\sum_{j=1}^{m}(\mathbb{E}_{j-1}-\mathbb{E}_j)D\left(\frac{\tau_{j-1}+\tau_j}{2}\right)
\label{eq:C}
\end{equation}
The accrual payment due on the losses is finally given by
\begin{equation}
s\cdot B = s\cdot\sum_{j=1}^{m}\frac{1}{2}(\tau_j - \tau_{j-1})(\mathbb{E}_{j-1}-\mathbb{E}_j)D(\frac{\tau_{j-1}+\tau_j}{2})
\label{eq:B}
\end{equation}

The value of the tranche, valued from the point of view of the protection buyer is $C-sA-sB$. The breakeven spread 
on the tranche occurs when the present value of the payments equals the present value of the payoffs so

$$ s = \cfrac{C}{A+B}$$

Suppose that the tranche under consideration covers losses on the portfolio between $\alpha_L$ and $\alpha_H$ and define

$$n_L = \cfrac{\alpha_L n}{1-R}\qquad \mathrm{and}\qquad n_H = \cfrac{\alpha_H n}{1-R}$$
where $R$ is the recovery rate. Finally define $m(x)$ as the smallest integer greater than $x$.
By definition the tranche principal stays $N$ while the number of defaults $k$ is less than $m(n_L)$, it is zero when the number of default is greater or equal to $m(n_H)$, otherwise is

$$\cfrac{\alpha_H -k(1-R)/n}{\alpha_H - \alpha_L}$$

The expected tranche principal at time $\tau_j$ conditional of the value of the factor $M$ is
$$\begin{equation}
\mathbb{E}_j(M) = \sum_{k=0}^{m(n_L)-1}\mathbb{P}(k, \tau_j|M) + \sum_{k=m(n_L)}^{m(n_H)-1}\mathbb{P}(k, \tau_j|M) \cfrac{\alpha_H -k(1-R)/n}{\alpha_H - \alpha_L}
\label{eq:E}
\end{equation}
$$

To compute the breakeven spread it is necessary to substitute Eq.~\ref{eq:E} into Eq.~\ref{eq:A},~\ref{eq:B} and~\ref{eq:C}
and we need to integrate the result over the variable $M$ (remember that has a standard normal distribution). 
The integration is quite complicated and is best accomplished with a technique called *Gaussian quadrature* which exploits the approximation

$$\int_{-\infty}^{\infty}{\cfrac{1}{\sqrt{2}}e^{-M^{2}/2}g(M)dM} \approx \sum_{k=1}^{k=L}w_k g(F_k)$$
as $L$ increases, accuracy increases.


## Complex Correlation Structures and the Financial Crisis

In the example above we have used the multivariate normal which gave rise to the Gaussian copula.However, we can use other and more complex copulas as well. For example we might want to assume the correlation is non-symmetruc which is useful in quant finance where correlations become very strong during market crashes and returns very negative.

Infact, Gaussian copulas are said to have played a key role in the 2008 financial crisis as tail-correlations were severely underestimated. Consider a set of mortgages in CDOs (a particular kind of contract that we are going to see) they are clearly correlated, if one mortgage fails, the likelihood that another failing is increased. In the early 2000s, the banks only knew how to model the marginals of the default rates. An (in)famous paper by Li then suggested to use copulas to model the correlations between those marginals. Rating agencies relied on thid model so heaviy, severely underestimating risk and giving false ratings...

If you are interested in the argument read \href{http://samueldwatts.com/wp-content/uploads/2016/08/Watts-Gaussian-Copula_Financial_Crisis.pdf}{this paper} for an excellent description of Gaussian copulas and the Financial Crisis which argues that different copula choices would not have made a difference but instead the assumed correlation was way too low.